Extract Data

In [1]:
import requests
import polars as pl
import pandas as pd
from zipfile import ZipFile

In [2]:
DATA_URL = "https://s3.amazonaws.com/capitalbikeshare-data/202212-capitalbikeshare-tripdata.zip"

# pl.read_csv(
#     ZipFile(DATA_URL ).open("my_file.csv", method='r').read()
# )# 

In [3]:
# pd.read_csv(DATA_URL).head()


In [4]:
from datetime import datetime

time_stamp = datetime.now().strftime("%d-%m-%Y")

response = requests.get(DATA_URL)

with open(f"{time_stamp}_capitalbikeshare-tripdata.zip", 'wb') as f:
    f.write(response.content)

In [ ]:
from zipfile import ZipFile
import os

folder_name = f"{time_stamp}_capitalbikeshare-tripdata"

folder = os.makedirs(folder_name, exist_ok=True)


with ZipFile(f"{time_stamp}_capitalbikeshare-tripdata.zip", "r") as zip_ref:
    zip_ref.extractall(folder_name)

In [6]:
def extract_file():
    for file in os.listdir(folder_name):
        if file.endswith(".csv"):
            return os.path.join(folder_name,file)

Transform

In [7]:
DATA_PATH = extract_file()
print(DATA_PATH)

30-04-2025_capitalbikeshare-tripdata\202212-capitalbikeshare-tripdata.csv


In [8]:
data = pl.read_csv(DATA_PATH)
data.head()

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
str,str,str,str,str,i64,str,i64,f64,f64,f64,f64,str
"""1C109AA07D71B300""","""classic_bike""","""2022-12-02 11:59:29""","""2022-12-02 12:02:44""","""17th St & Massachusetts Ave NW""",31267,"""Massachusetts Ave & Dupont Cir…",31200,38.908142,-77.038359,38.9101,-77.0444,"""member"""
"""DD4F1F66ACD60BDA""","""classic_bike""","""2022-12-14 19:09:35""","""2022-12-14 19:47:14""","""C & O Canal & Wisconsin Ave NW""",31225,"""8th & F St NE""",31631,38.90375,-77.06269,38.897274,-76.994749,"""member"""
"""B109378ACEA4F614""","""classic_bike""","""2022-12-03 14:21:50""","""2022-12-03 14:23:06""","""Potomac Ave & 35th St S""",31052,"""S Clark St & 33rd St""",31944,38.844015,-77.050537,38.845028,-77.051956,"""member"""
"""BC5B3C71DDD1A782""","""classic_bike""","""2022-12-19 14:28:28""","""2022-12-19 14:39:47""","""Market Square / King St & Roya…",31042,"""Potomac Greens Dr & Slaters Ln""",31083,38.804718,-77.043363,38.82175,-77.047494,"""member"""
"""ED7903CC0350847E""","""classic_bike""","""2022-12-07 13:46:30""","""2022-12-07 13:49:20""","""Market Square / King St & Roya…",31042,"""King St & Patrick St""",31044,38.804718,-77.043363,38.805317,-77.049883,"""member"""


In [9]:
data.columns

['ride_id',
 'rideable_type',
 'started_at',
 'ended_at',
 'start_station_name',
 'start_station_id',
 'end_station_name',
 'end_station_id',
 'start_lat',
 'start_lng',
 'end_lat',
 'end_lng',
 'member_casual']

In [10]:
len(data)

175233

In [11]:
data.schema

Schema([('ride_id', String),
        ('rideable_type', String),
        ('started_at', String),
        ('ended_at', String),
        ('start_station_name', String),
        ('start_station_id', Int64),
        ('end_station_name', String),
        ('end_station_id', Int64),
        ('start_lat', Float64),
        ('start_lng', Float64),
        ('end_lat', Float64),
        ('end_lng', Float64),
        ('member_casual', String)])

In [ ]:
schema = {
    'ride_id': pl.String,
    'rideable_type': pl.String,
    'started_at': pl.String,
    'ended_at': pl.String,  
    'start_station_name': pl.String,
    'start_station_id': pl.Int64,
    'end_station_name': pl.String,
    'end_station_id': pl.Int64,
    'start_lat': pl.Float64,
    'start_lng': pl.Float64,
    'end_lat': pl.Float64,
    'end_lng': pl.Float64,
    'member_casual': pl.String
}

data = pl.DataFrame(data, schema=schema)

# # convert the string data time to datetime
# data = data.with_columns(
#     pl.col('started_at').str.to_datetime(),
#     pl.col('ended_at').str.to_datetime()
# )


# data = data.with_columns(
#     (pl.col('ended_at') - pl.col('started_at')).alias('duration')
# )

# data = data.with_columns(
#     pl.col('duration').dt.total_seconds().alias('duration_seconds')
# )


# data = data.with_columns(
#     pl.col('ended_at').dt.week().alias('week')
# )


df = data.with_columns([
    pl.col('started_at').str.to_datetime(),
    pl.col('ended_at').str.to_datetime(),
    (pl.col('ended_at').str.to_datetime() - pl.col('started_at').str.to_datetime()).alias('duration'),
    (pl.col('ended_at').str.to_datetime() - pl.col('started_at').str.to_datetime())
        .dt.total_seconds().alias('duration_seconds'),
    pl.col('ended_at').str.to_datetime().dt.week().alias('week')
])

df


ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration,duration_seconds,week
str,str,datetime[μs],datetime[μs],str,i64,str,i64,f64,f64,f64,f64,str,duration[μs],i64,i8
"""1C109AA07D71B300""","""classic_bike""",2022-12-02 11:59:29,2022-12-02 12:02:44,"""17th St & Massachusetts Ave NW""",31267,"""Massachusetts Ave & Dupont Cir…",31200,38.908142,-77.038359,38.9101,-77.0444,"""member""",3m 15s,195,48
"""DD4F1F66ACD60BDA""","""classic_bike""",2022-12-14 19:09:35,2022-12-14 19:47:14,"""C & O Canal & Wisconsin Ave NW""",31225,"""8th & F St NE""",31631,38.90375,-77.06269,38.897274,-76.994749,"""member""",37m 39s,2259,50
"""B109378ACEA4F614""","""classic_bike""",2022-12-03 14:21:50,2022-12-03 14:23:06,"""Potomac Ave & 35th St S""",31052,"""S Clark St & 33rd St""",31944,38.844015,-77.050537,38.845028,-77.051956,"""member""",1m 16s,76,48
"""BC5B3C71DDD1A782""","""classic_bike""",2022-12-19 14:28:28,2022-12-19 14:39:47,"""Market Square / King St & Roya…",31042,"""Potomac Greens Dr & Slaters Ln""",31083,38.804718,-77.043363,38.82175,-77.047494,"""member""",11m 19s,679,51
"""ED7903CC0350847E""","""classic_bike""",2022-12-07 13:46:30,2022-12-07 13:49:20,"""Market Square / King St & Roya…",31042,"""King St & Patrick St""",31044,38.804718,-77.043363,38.805317,-77.049883,"""member""",2m 50s,170,49
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""99A72D09AA13F4F9""","""classic_bike""",2022-12-29 11:50:13,2022-12-29 12:00:30,"""5th & K St NW""",31600,"""New Jersey Ave & F St NW""",31655,38.90304,-77.019027,38.897108,-77.011616,"""member""",10m 17s,617,52
"""12E5CC8D2DAE1507""","""classic_bike""",2022-12-05 19:14:05,2022-12-05 19:22:10,"""5th & K St NW""",31600,"""New Jersey Ave & F St NW""",31655,38.90304,-77.019027,38.897108,-77.011616,"""casual""",8m 5s,485,49
"""207B3857E702843E""","""electric_bike""",2022-12-05 12:51:38,2022-12-05 12:56:16,"""5th & K St NW""",31600,"""New Jersey Ave & F St NW""",31655,38.903068,-77.018793,38.897108,-77.011616,"""casual""",4m 38s,278,49


In [ ]:
df = data.unique(subset='ride_id')
df

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration,duration_seconds,week
str,str,datetime[μs],datetime[μs],str,i64,str,i64,f64,f64,f64,f64,str,duration[μs],i64,i8
"""F208903A9962E502""","""electric_bike""",2022-12-14 18:45:40,2022-12-14 19:13:06,"""16th & Harvard St NW""",31135,"""10th & E St NW""",31256,38.92615,-77.036683,38.895914,-77.026064,"""member""",27m 26s,1646,50
"""DF3D6AD7D3BBB5DB""","""electric_bike""",2022-12-11 16:00:46,2022-12-11 16:09:55,"""Fort Totten Metro""",31515,"""5th & Kennedy St NW""",31403,38.952357,-77.002675,38.956556,-77.019814,"""casual""",9m 9s,549,49
"""82B36099FD06BC1B""","""electric_bike""",2022-12-16 15:34:09,2022-12-16 15:53:04,"""Virginia Theological Seminary""",31964,"""Henry St & Pendleton St""",31046,38.820114,-77.09398,38.811456,-77.050276,"""casual""",18m 55s,1135,50
"""7D1719E64BBCFEBA""","""classic_bike""",2022-12-23 12:36:22,2022-12-23 12:39:42,"""Georgetown Harbor / 30th St NW""",31215,"""Kennedy Center""",31211,38.902314,-77.059141,38.897293,-77.05557,"""member""",3m 20s,200,51
"""9FDD9F9F194C5AE8""","""classic_bike""",2022-12-05 15:05:46,2022-12-05 15:18:30,"""Reservoir Rd & 38th St NW""",31325,"""Connecticut Ave & R St NW""",31299,38.912614,-77.074323,38.912644,-77.04564,"""member""",12m 44s,764,49
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""01B1D9691B663046""","""electric_bike""",2022-12-27 14:31:34,2022-12-27 14:37:19,"""4th & D St NW / Judiciary Squa…",31621,"""Constitution Ave & 2nd St NW/D…",31271,38.895579,-77.01615,38.892275,-77.013917,"""casual""",5m 45s,345,52
"""86FD01AB08E140A6""","""electric_bike""",2022-12-05 15:30:40,2022-12-05 15:44:35,"""7th & S St NW""",31130,"""M St & Pennsylvania Ave NW""",31246,38.91424,-77.021633,38.905126,-77.056887,"""member""",13m 55s,835,49
"""75B1A97E743BC981""","""docked_bike""",2022-12-30 12:11:04,2022-12-30 12:54:08,"""Kennedy Center""",31211,"""4th St & Madison Dr NW""",31288,38.897293,-77.05557,38.890496,-77.017247,"""casual""",43m 4s,2584,52


In [ ]:
data.select(pl.col('rideable_type').value_counts()).unnest('rideable_type')

rideable_type,count
str,u32
"""electric_bike""",29199
"""classic_bike""",140796
"""docked_bike""",5238


In [ ]:
data.select(pl.col('start_station_name').value_counts()).unnest('start_station_name')

start_station_name,count
str,u32
"""Garland Ave & Walden Rd""",12
"""King Farm Blvd & Pleasant Dr""",17
"""37th & Ely Pl SE""",4
"""15th & K St NW""",440
"""E Fairfax St & S Washington St""",15
…,…
"""S Four Mile Run Dr & Shirlingt…",19
"""Frederick Ave & Horners Ln""",5
"""2nd St & Massachusetts Ave NE""",724


In [ ]:
data.select(pl.col('member_casual').value_counts()).unnest('member_casual')

member_casual,count
str,u32
"""member""",121020
"""casual""",54213


In [ ]:
#3 flag data with 45mins ride and above (45min = 2700secs)
# - A casual rider starts a trip at midnight
# - Any ride lasts longer than 45 minutes
import logging

# Setup logging to a file
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)
file_handler = logging.FileHandler('ride_flags.log')
formatter = logging.Formatter('%(asctime)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

def log_flagged_data(func):
    """log flagged data"""
    def wrapper(*args, **kwargs):
        flagged_data = func(*args, **kwargs)
        logger.warning(f"Logged flagged data: {flagged_data} rows")
        yield flagged_data
    return wrapper

@log_flagged_data
def flag_longer_ride(df, col_name):
    """Flag longer rides (over 27,000 seconds, ~45 minutes)"""
    flagged_df = df.filter(pl.col(col_name) > 27000)
    print(flagged_df)
    yield logger.warning(f"Flagged rides with duration greater than 45 minutes: {flagged_df} rows")

@log_flagged_data
def flag_later_hour_ride(df, col_name):
    """Flag rides that occurred later than 23:30:00"""
    flagged_df = df.filter(pl.col(col_name).dt.hour() >=23 & pl.col(col_name).dt.minute() >=59)
    flagged_df.collect()
    yield logger.warning(f"Flagged rides later than 23:30:00:{flagged_df} rows")


for df in flag_later_hour_ride(data,"ended_at"):
    df

for df in flag_longer_ride(df, "duration_seconds"):
    df

Partitioned Data using polars

In [ ]:
data.write_parquet(f'capitalbikeshare-tripdata_{time_stamp}.parquet',
    partition_by=['member_casual','week']
)

In [ ]:
def flag_longer_ride(df, col_name):
    """Flag longer rides (over 27,000 seconds, ~45 minutes)"""
    flagged_df = df.filter(pl.col(col_name) > 27000)
    yield flagged_df.show()
    # yield logger.warning(f"Flagged rides with duration greater than 45 minutes: {flagged_df} rows")

flag_later_hour_ride(data,"ended_at") 

<generator object log_flagged_data.<locals>.wrapper at 0x00000211044853F0>

In [ ]:
data.head()

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration,duration_seconds,week
str,str,datetime[μs],datetime[μs],str,i64,str,i64,f64,f64,f64,f64,str,duration[μs],i64,i8
"""1C109AA07D71B300""","""classic_bike""",2022-12-02 11:59:29,2022-12-02 12:02:44,"""17th St & Massachusetts Ave NW""",31267,"""Massachusetts Ave & Dupont Cir…",31200,38.908142,-77.038359,38.9101,-77.0444,"""member""",3m 15s,195,48
"""DD4F1F66ACD60BDA""","""classic_bike""",2022-12-14 19:09:35,2022-12-14 19:47:14,"""C & O Canal & Wisconsin Ave NW""",31225,"""8th & F St NE""",31631,38.90375,-77.06269,38.897274,-76.994749,"""member""",37m 39s,2259,50
"""B109378ACEA4F614""","""classic_bike""",2022-12-03 14:21:50,2022-12-03 14:23:06,"""Potomac Ave & 35th St S""",31052,"""S Clark St & 33rd St""",31944,38.844015,-77.050537,38.845028,-77.051956,"""member""",1m 16s,76,48
"""BC5B3C71DDD1A782""","""classic_bike""",2022-12-19 14:28:28,2022-12-19 14:39:47,"""Market Square / King St & Roya…",31042,"""Potomac Greens Dr & Slaters Ln""",31083,38.804718,-77.043363,38.82175,-77.047494,"""member""",11m 19s,679,51
"""ED7903CC0350847E""","""classic_bike""",2022-12-07 13:46:30,2022-12-07 13:49:20,"""Market Square / King St & Roya…",31042,"""King St & Patrick St""",31044,38.804718,-77.043363,38.805317,-77.049883,"""member""",2m 50s,170,49


In [ ]:
data.filter(pl.col("ended_at") > 27000)

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration,duration_seconds,week
str,str,datetime[μs],datetime[μs],str,i64,str,i64,f64,f64,f64,f64,str,duration[μs],i64,i8
"""1C109AA07D71B300""","""classic_bike""",2022-12-02 11:59:29,2022-12-02 12:02:44,"""17th St & Massachusetts Ave NW""",31267,"""Massachusetts Ave & Dupont Cir…",31200,38.908142,-77.038359,38.9101,-77.0444,"""member""",3m 15s,195,48
"""DD4F1F66ACD60BDA""","""classic_bike""",2022-12-14 19:09:35,2022-12-14 19:47:14,"""C & O Canal & Wisconsin Ave NW""",31225,"""8th & F St NE""",31631,38.90375,-77.06269,38.897274,-76.994749,"""member""",37m 39s,2259,50
"""B109378ACEA4F614""","""classic_bike""",2022-12-03 14:21:50,2022-12-03 14:23:06,"""Potomac Ave & 35th St S""",31052,"""S Clark St & 33rd St""",31944,38.844015,-77.050537,38.845028,-77.051956,"""member""",1m 16s,76,48
"""BC5B3C71DDD1A782""","""classic_bike""",2022-12-19 14:28:28,2022-12-19 14:39:47,"""Market Square / King St & Roya…",31042,"""Potomac Greens Dr & Slaters Ln""",31083,38.804718,-77.043363,38.82175,-77.047494,"""member""",11m 19s,679,51
"""ED7903CC0350847E""","""classic_bike""",2022-12-07 13:46:30,2022-12-07 13:49:20,"""Market Square / King St & Roya…",31042,"""King St & Patrick St""",31044,38.804718,-77.043363,38.805317,-77.049883,"""member""",2m 50s,170,49
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""99A72D09AA13F4F9""","""classic_bike""",2022-12-29 11:50:13,2022-12-29 12:00:30,"""5th & K St NW""",31600,"""New Jersey Ave & F St NW""",31655,38.90304,-77.019027,38.897108,-77.011616,"""member""",10m 17s,617,52
"""12E5CC8D2DAE1507""","""classic_bike""",2022-12-05 19:14:05,2022-12-05 19:22:10,"""5th & K St NW""",31600,"""New Jersey Ave & F St NW""",31655,38.90304,-77.019027,38.897108,-77.011616,"""casual""",8m 5s,485,49
"""207B3857E702843E""","""electric_bike""",2022-12-05 12:51:38,2022-12-05 12:56:16,"""5th & K St NW""",31600,"""New Jersey Ave & F St NW""",31655,38.903068,-77.018793,38.897108,-77.011616,"""casual""",4m 38s,278,49
